In [ ]:
import pandas as pd
import bs4 as bs
import requests
import re
import us

STATES_DICT = us.states.mapping('name', 'abbr')
PARTY_DICT = {'republican' : 'r', 'democratic' : 'd', 'independent': 'i'}

# Governor

### Governor 2010-2013

In [ ]:
def scrape_governor_names_(year):
    RACE_TYPE = 'governor'
    source = requests.get(f'https://en.wikipedia.org/wiki/United_States_gubernatorial_elections,_{year}')
    soup = bs.BeautifulSoup(source.content, features='html.parser')
    
    names = []
    parties = []
    race_types = []
    years = []
    states = []
    
    for table in soup.find_all('table', class_="wikitable sortable"):
        if all(['candidates' not in h.text.strip().lower() for h in table.find_all('th', text=True)]):
            continue
            
        for tr in table.find_all('tr')[1:]:
            cells = tr.find_all('td')
            state = cells[0].text.strip()
            
            # incumbent
            incumbent_name = cells[1].text.lower().strip()
            incumbent_name = 
            incumbent_party = cells[2].text.lower().strip()
            names.append(incumbent_name)
            parties.append(PARTY_DICT[incumbent_party.lower()] if incumbent_party.lower() in PARTY_DICT.keys() else incumbent_party.lower())
            race_types.append(RACE_TYPE)
            years.append(year)
            states.append(state)
            
            # competitors
            candidates = cells[-1]
            candidates_text = re.findall(r"\w[\s\w ]+?\(\w+?\)", candidates.text)
            for candidate in candidates_text:
                matcher = re.match(r"([\w\s]+)\s\((\w+)\)", candidate.lower())
                if matcher:
                    name, party = matcher.groups()
                    names.append(name.strip())
                    parties.append(party)
                    race_types.append(RACE_TYPE)
                    years.append(year)
                    states.append(state)

    # return df
    df = pd.DataFrame({'name': names, 
                       'party': parties,
                       'race_type': race_types,
                       'year': years,
                       'state': states})
    df['state'] = df['state'].map(STATES_DICT)
#     df['party'] = df['party'].map(PARTY_DICT)
    df.loc[~df['party'].isin(['d', 'r']), 'party'] = 'o'
    
    return df